In [13]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# keras imports
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json

In [12]:
# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import pickle
import datetime
import time

In [3]:
import tensorflow as tf
tf.__version__

'1.3.0'

In [4]:
!jupyter kernelspec list
import sys
sys.executable

Available kernels:
  python3    /home/xiaoran/anaconda3/envs/Rita36/share/jupyter/kernels/python3


'/home/xiaoran/anaconda3/envs/Rita36/bin/python'

In [5]:
# filter warnings
# load the user configs
with open('conf/conf_xception.json') as f:    
    config = json.load(f)
# config variables
model_name = config["model"]
weights = config["weights"]
include_top = config["include_top"]
train_path = config["train_path"]
features_path = config["features_path"]
labels_path = config["labels_path"]
test_size = config["test_size"]
results = config["results"]
model_path = config["model_path"]
#weights_path = config["weights_path"]
print('model_name {}. features_path {}'.format(model_name, features_path))

model_name xception. features_path output/flowers_17/xception/features.h5


In [6]:
# start time
print("[STATUS] start time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))
start = time.time()

# create the pretrained models
# check for pretrained weight usage or not
# check for top layers to be included or not

if model_name == "vgg16":
    base_model = VGG16(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
    image_size = (224, 224)
elif model_name == "vgg19":
    base_model = VGG19(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
    image_size = (224, 224)
elif model_name == "resnet50":
    base_model = ResNet50(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('flatten').output)
    image_size = (224, 224)
elif model_name == "inceptionv3":
    base_model = InceptionV3(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output) # this is correct
    #model = Model(input=base_model.input, output=base_model.get_layer('Dense').output)
    image_size = (299, 299)
elif model_name == "xception":
    base_model = Xception(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output)
    image_size = (299, 299)
else:
    base_model = None

print("[INFO] successfully loaded base model and model...")

[STATUS] start time - 2017-11-16 17:20
[INFO] successfully loaded base model and model...


/home/xiaoran/anaconda3/envs/Rita36/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`


In [30]:
# path to training dataset
train_files = os.listdir(train_path)

# encode the labels
#print("[INFO] encoding labels...")
#le = LabelEncoder()
#le.fit([tl for tl in train_labels])
# loop over all the labels in the folder
with open('/home/xiaoran/Dropbox/preTrain/flower_recognition/dataset/flowers_17/names.txt') as f:
    nn = f.readlines()
    names = [x.strip() for x in nn] 
len(names)

1360

In [31]:
num = 0 
label = 0
# variables to hold features and labels
features = [] # store feature data, x
labels   = [] # store label data, y
for name in names:
    img = cv2.imread('/home/xiaoran/Dropbox/preTrain/flower_recognition/dataset/flowers_17/'+name)
#     fd = img.flatten()
#     im_flt.append(fd)
#     imHog_list.append(img)
#     if num %10 == 0:
#         print(' {}'.format(i), end=',')
    if num % 80 == 0 :
        label = label+1
#         print('\n file {}, label {}'.format(file, label))    
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    features.append(flat)
    labels.append(label)
    num = num + 1

# print("[INFO] processed - {}".format(i))
print( "[INFO] completed label - {}".format(label))
print(len(labels))

[INFO] processed - 1045
[INFO] completed label - 17
1360


In [22]:
# num = [10,20,80,160,90]
# label = 0
# labels = []
# for n in num:
#     if n % 80 == 0 :
#         label = label+1
#         print(n)
#     labels.append(label)
# labels

80
160


[0, 0, 1, 2, 2]

In [32]:
print('features shape is ', np.shape(features))
print('feature shape is ', np.shape(feature))
print('flat shape is ', np.shape(flat))

features shape is  (1360, 2048)
feature shape is  (1, 2048)
flat shape is  (2048,)


In [34]:
# encode the labels using LabelEncoder
targetNames = np.unique(labels)
#le = LabelEncoder()
#le_labels = le.fit_transform(labels)
le_labels = np.array(labels)
# get the shape of training labels
print("[STATUS] training labels: {}".format(le_labels))
print("[STATUS] training labels shape: {}".format(le_labels.shape))

# save features and labels
h5f_data = h5py.File(features_path, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File(labels_path, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

h5f_data.close()
h5f_label.close()

# save model and weights
model_json = model.to_json()
with open(model_path + str(test_size) + ".json", "w") as json_file:
    json_file.write(model_json)

# save weights
model.save_weights(model_path + str(test_size) + ".h5")
print("[STATUS] saved model and weights to disk..")

print("[STATUS] features and labels saved..")

# end time
end = time.time()
print("[STATUS] end time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))

[STATUS] training labels: [ 1  1  1 ..., 17 17 17]
[STATUS] training labels shape: (1360,)
[STATUS] saved model and weights to disk..
[STATUS] features and labels saved..
[STATUS] end time - 2017-11-17 09:49
